In [1]:
!pip install --upgrade pip
!pip install numpy==1.18
!pip install mediapipe-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.18.0.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
!wget https://storage.googleapis.com/mediapipe-tasks/object_detector/android_figurine.zip
!unzip android_figurine.zip
train_dataset_path = "android_figurine/train"
validation_dataset_path = "android_figurine/validation"

--2023-06-12 12:55:34--  https://storage.googleapis.com/mediapipe-tasks/object_detector/android_figurine.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 142.250.107.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28607322 (27M) [application/zip]
Saving to: ‘android_figurine.zip’

android_figurine.zi 100%[===================>]  27.28M   155MB/s    in 0.2s    

2023-06-12 12:55:34 (155 MB/s) - ‘android_figurine.zip’ saved [28607322/28607322]

Archive:  android_figurine.zip
   creating: android_figurine/
   creating: android_figurine/validation/
  inflating: android_figurine/validation/labels.json  
   creating: android_figurine/validation/images/
  inflating: android_figurine/validation/images/IMG_0502.jpg  
  inflating: android_figurine/validation/images/IMG_0497.jpg  
  inflating: android_figurine/validation/images/IMG_0501.jpg 

In [5]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

0: background
1: android
2: pig_android


In [6]:
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
from collections import defaultdict
import math

def draw_outline(obj):
  obj.set_path_effects([patheffects.Stroke(linewidth=4,  foreground='black'), patheffects.Normal()])
def draw_box(ax, bb):
  patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
  draw_outline(patch)
def draw_text(ax, bb, txt, disp):
  text = ax.text(bb[0],(bb[1]-disp),txt,verticalalignment='top'
  ,color='white',fontsize=10,weight='bold')
  draw_outline(text)
def draw_bbox(ax, annotations_list, id_to_label, image_shape):
  for annotation in annotations_list:
    cat_id = annotation["category_id"]
    bbox = annotation["bbox"]
    draw_box(ax, bbox)
    draw_text(ax, bbox, id_to_label[cat_id], image_shape[0] * 0.05)
def visualize(dataset_folder, max_examples=None):
  with open(os.path.join(dataset_folder, "labels.json"), "r") as f:
    labels_json = json.load(f)
  images = labels_json["images"]
  cat_id_to_label = {item["id"]:item["name"] for item in labels_json["categories"]}
  image_annots = defaultdict(list)
  for annotation_obj in labels_json["annotations"]:
    image_id = annotation_obj["image_id"]
    image_annots[image_id].append(annotation_obj)

  if max_examples is None:
    max_examples = len(image_annots.items())
  n_rows = math.ceil(max_examples / 3)
  fig, axs = plt.subplots(n_rows, 3, figsize=(24, n_rows*8)) # 3 columns(2nd index), 8x8 for each image
  for ind, (image_id, annotations_list) in enumerate(list(image_annots.items())[:max_examples]):
    ax = axs[ind//3, ind%3]
    img = plt.imread(os.path.join(dataset_folder, "images", images[image_id]["file_name"]))
    ax.imshow(img)
    draw_bbox(ax, annotations_list, cat_id_to_label, img.shape)
  plt.show()

visualize(train_dataset_path, 9)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


train_data size:  62
validation_data size:  10


In [8]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model', batch_size=10, epochs=10 )
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [9]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32)  3704416   
                             , '3': (None, 32, 32, 64            
                             ),                                  
                              '4': (None, 16, 16, 160            
                             ),                                  
                              '5': (None, 8, 8, 192),            
                              '6': (None, 1, 1, 1280)            
                             }                                   
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128),  144928    
                              '4': (None, 16, 16, 128            
                             ),                                  
                              '3': (None, 32, 32, 

/usr/local/lib/python3.10/dist-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


6/6 [==============================] - 97s 7s/step - total_loss: 6.1320 - cls_loss: 5.9525 - box_loss: 0.0023 - model_loss: 6.0691 - val_total_loss: 1.8279 - val_cls_loss: 1.7024 - val_box_loss: 0.0012 - val_model_loss: 1.7648
Epoch 2/10
6/6 [==============================] - 30s 5s/step - total_loss: 2.0397 - cls_loss: 1.8679 - box_loss: 0.0022 - model_loss: 1.9764 - val_total_loss: 1.5450 - val_cls_loss: 1.3970 - val_box_loss: 0.0017 - val_model_loss: 1.4814
Epoch 3/10
6/6 [==============================] - 33s 5s/step - total_loss: 1.4540 - cls_loss: 1.3170 - box_loss: 0.0015 - model_loss: 1.3903 - val_total_loss: 1.2539 - val_cls_loss: 1.0915 - val_box_loss: 0.0020 - val_model_loss: 1.1899
Epoch 4/10
6/6 [==============================] - 30s 5s/step - total_loss: 1.1390 - cls_loss: 1.0222 - box_loss: 0.0011 - model_loss: 1.0748 - val_total_loss: 1.1156 - val_cls_loss: 0.9382 - val_box_loss: 0.0023 - val_model_loss: 1.0512
Epoch 5/10
6/6 [==============================] - 31s 5s/st

In [10]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

3/3 [==============================] - 6s 1s/step - total_loss: 0.6129 - cls_loss: 0.4700 - box_loss: 0.0016 - model_loss: 0.5481
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.12s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.694
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.977
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.740
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.694
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.749
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.749
 Average Recall     (AR) @[ IoU

In [11]:
model.evaluate(validation_data)

10/10 [==============================] - 5s 384ms/step - total_loss: 0.5961 - cls_loss: 0.4585 - box_loss: 0.0015 - model_loss: 0.5312
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.694
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.977
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.740
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.694
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.749
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.749
 Average Recall     (AR) @

([0.5960677862167358,
  0.45852479338645935,
  0.0014540597330778837,
  0.5312277674674988],
 {'AP': 0.6936025,
  'AP50': 0.9770977,
  'AP75': 0.740099,
  'APs': -1.0,
  'APm': -1.0,
  'APl': 0.6936025,
  'ARmax1': 0.74930555,
  'ARmax10': 0.74930555,
  'ARmax100': 0.74930555,
  'ARs': -1.0,
  'ARm': -1.0,
  'ARl': 0.74930555})

In [12]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>